In [1]:
import numpy as np
import cv2 as cv
import time
import numpy as np
from deepface import DeepFace

2023-10-28 14:30:02.047627: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-28 14:30:02.142467: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [8]:
# Define backends and models
backends = [
  'retinaface', 
]

In [9]:
def draw(image, bounding_boxes, text):
    i = 1
    for bbox in bounding_boxes:
        x = bbox['region']['x']
        y = bbox['region']['y']
        w = bbox['region']['w']
        h = bbox['region']['h']

        # Draw a rectangle on the image
        img = cv.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)  # (0, 255, 0) is the color in BGR format
        img = cv.putText(img, text, (x, y - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        i+=1
    return img 

In [10]:
def resize_image(srcimg,input_height, input_width, keep_ratio=True):
    if keep_ratio and srcimg.shape[0] != srcimg.shape[1]:
        hw_scale = srcimg.shape[0] / srcimg.shape[1]
        if hw_scale > 1:
            newh, neww = input_height, int(input_width / hw_scale)
            img = cv.resize(srcimg, (neww, newh), interpolation=cv.INTER_AREA)
        else:
            newh, neww = int(input_height * hw_scale), input_width
            img = cv.resize(srcimg, (neww, newh), interpolation=cv.INTER_AREA)
    else:
        img = cv.resize(srcimg, (input_width, input_height), interpolation=cv.INTER_AREA)
    return img 

In [12]:
video_file_path = 'experimentation_videos/videoplayback.mp4'  # Replace with the path to your video file
for i, backend in enumerate(backends):
    cap = cv.VideoCapture(video_file_path)
    fps = cap.get(cv.CAP_PROP_FPS)
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fourcc = cv.VideoWriter_fourcc(*'XVID')
    out = cv.VideoWriter('model_output/experiment_4_{i}.avi',cv.VideoWriter_fourcc('M','J','P','G'), fps, (frame_width,frame_height))
    if not cap.isOpened():
        print("Error: Could not open video file.")

    start_time = time.time()

    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        frame = resize_image(frame, frame_height, frame_width)
        objs = DeepFace.analyze(frame,
                actions = ['emotion'],
                enforce_detection=False,
                silent=True,
                detector_backend=backend
        )


        frame = draw(frame, objs, objs[0]["dominant_emotion"])
        out.write(frame)
        # cv.imshow("Img", frame)

        if cv.waitKey(1) & 0xFF == ord('q'):
            break
            
    cap.release()
    out.release() # Release the video file

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"For {backend}, Elapsed time: {elapsed_time} seconds")


2023-10-28 14:33:17.426020: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401
2023-10-28 14:33:23.497228: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


For retinaface, Elapsed time: 1977.4030644893646 seconds


In [ ]:
#Retinaface takes about 32 minutes on my system